модель пс
модель бс
тест что инвариантны
тест что воспроизводят либраскаловскую штуковину

In [1]:
'''!wget "https://archive.materialscloud.org/record/file?file_id=b612d8e3-58af-4374-96ba-b3551ac5d2f4&filename=methane.extxyz.gz&record_id=528" -O methane.extxyz.gz
!gunzip -k methane.extxyz.gz'''

'!wget "https://archive.materialscloud.org/record/file?file_id=b612d8e3-58af-4374-96ba-b3551ac5d2f4&filename=methane.extxyz.gz&record_id=528" -O methane.extxyz.gz\n!gunzip -k methane.extxyz.gz'

In [2]:
import numpy as np
from rascal.representations import SphericalExpansion as SPH
import ase.io
import tqdm
from nice.blocks import *
from nice.utilities import *
from matplotlib import pyplot as plt
from sklearn.linear_model import BayesianRidge
import torch
torch.set_num_threads(1)
import torch.nn
from code_pytorch import *
import time
import numpy as np
from sympy.physics.wigner import wigner_3j
from sympy.physics.quantum.spin import Rotation as rot
from rascal.representations import SphericalInvariants as SOAP
from torch2trt import torch2trt

In [3]:
LAMBDA_MAX = 5

In [4]:
HYPERS = {
    'interaction_cutoff': 6.3,
    'max_radial': 5,
    'max_angular': LAMBDA_MAX,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.3,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO'
}

In [5]:
structures = ase.io.read('methane.extxyz', index='0:300')

In [6]:
all_species = get_all_species(structures)

coefficients = get_spherical_expansion(structures, HYPERS,
                                             all_species)

print(coefficients.keys())
print(coefficients[1].shape)

100%|██████████| 2/2 [00:00<00:00, 700.28it/s]

dict_keys([1, 6])
(1200, 10, 6, 11)


In [7]:
def convert_to_torch(coefficients):
    result = {}
    for lambd in range(coefficients.shape[2]):
        result[lambd] = torch.FloatTensor(coefficients[:, :, lambd, : 2 * lambd + 1])
        result[lambd] = result[lambd].transpose(0, -1)
    return result

In [8]:
coefficients[1] = convert_to_torch(coefficients[1])
coefficients[6] = convert_to_torch(coefficients[6])
print(coefficients[1][3].shape)

torch.Size([7, 10, 1200])


In [9]:
clebsch = nice.clebsch_gordan.ClebschGordan(LAMBDA_MAX)

In [10]:
class Powerspectrum(torch.nn.Module):
    def __init__(self, clebsch):
        super(Powerspectrum, self).__init__()
        self.first = ClebschCombining(clebsch, 0)       
            
    def forward(self, X):
        ps_invariants = self.first(X, X)       
        return ps_invariants

In [11]:
model_ps = Powerspectrum(clebsch.precomputed_)

In [12]:
result = model_ps(coefficients[6])
print(result.keys())
print(result[0].shape)

dict_keys([0])
torch.Size([1, 600, 300])


In [13]:
class Bispectrum(torch.nn.Module):
    def __init__(self, clebsch, lambda_max):
        super(Bispectrum, self).__init__()
        self.first = ClebschCombining(clebsch, lambda_max)
        self.second = ClebschCombining(clebsch, 0)
            
    def forward(self, X):
        ps_covariants = self.first(X, X)
        bs_invariants = self.second(ps_covariants, X)
        return bs_invariants

In [14]:
model_bs = Bispectrum(clebsch.precomputed_, LAMBDA_MAX)

In [15]:
result = model_bs(coefficients[6])

In [16]:
print(result.keys())
print(result[0].shape)

dict_keys([0])
torch.Size([1, 111000, 300])


In [17]:
class Trispectrum(torch.nn.Module):
    def __init__(self, clebsch, lambda_max):
        super(Trispectrum, self).__init__()
        self.first = ClebschCombining(clebsch, lambda_max)
        self.second = ClebschCombining(clebsch, lambda_max)
        self.third = ClebschCombining(clebsch, 0)
            
    def forward(self, X):
        ps_covariants = self.first(X, X)
        bs_covariants = self.second(ps_covariants, X)
        ts_invariants = self.third(bs_covariants, X)
        return ts_invariants

In [18]:
model_ts = Trispectrum(clebsch.precomputed_, LAMBDA_MAX)

In [19]:
small = {key : coefficients[6][key][:, :, 0:2] for key in coefficients[6].keys()}
result = model_ts(small)

In [20]:
print(result.keys())
print(result[0].shape)

dict_keys([0])
torch.Size([1, 22950000, 2])


In [21]:
'''model = ClebschCombining(clebsch.precomputed_, 2)
result = model(coefficients[6], coefficients[6])
for key in result.keys():
    print(key, result[key].shape)'''

'model = ClebschCombining(clebsch.precomputed_, 2)\nresult = model(coefficients[6], coefficients[6])\nfor key in result.keys():\n    print(key, result[key].shape)'

In [22]:
HYPERS_PS = copy.deepcopy(HYPERS)
HYPERS_PS['soap_type'] = 'PowerSpectrum'

In [23]:
soap_ps = SOAP(**HYPERS_PS)
begin = time.time()
ps = soap_ps.transform(nice.rascal_coefficients.process_structures(structures)).get_features(soap_ps)
end = time.time()
print(end - begin)

0.09392023086547852


In [24]:
print(ps.shape)

(1500, 450)


In [25]:
begin = time.time()
result = model_ps(coefficients[6])
result = model_ps(coefficients[1])
end = time.time()
print("time: ", end - begin)

time:  0.01616072654724121


In [26]:
HYPERS_BS = copy.deepcopy(HYPERS)
HYPERS_BS['soap_type'] = 'BiSpectrum'
HYPERS_BS['inversion_symmetry'] = False

In [27]:
soap_bs = SOAP(**HYPERS_BS)
begin = time.time()
bs = soap_bs.transform(nice.rascal_coefficients.process_structures(structures)).get_features(soap_bs)
end = time.time()
print(end - begin)
print(bs.shape)

100.31233167648315
(1500, 55500)


In [28]:
begin = time.time()
with torch.no_grad():
    result = model_bs(coefficients[1])
    result = model_bs(coefficients[6])
end = time.time()
print(end - begin)
print(result[0].shape)

4.656926155090332
torch.Size([1, 111000, 300])


In [29]:
#model_traced = torch.jit.script(model_bs, coefficients[1])

In [30]:
'''begin = time.time()
result = model_ts(coefficients[6])
end = time.time()
print(end - begin)
print(result[0].shape)'''

'begin = time.time()\nresult = model_ts(coefficients[6])\nend = time.time()\nprint(end - begin)\nprint(result[0].shape)'

In [31]:
for key in coefficients[1].keys():
    coefficients[1][key] = coefficients[1][key].cuda()
for key in coefficients[6].keys():
    coefficients[6][key] = coefficients[6][key].cuda()

In [32]:
model_ps = model_ps.cuda()

In [33]:
for _ in range(10):
    begin = time.time()
    result = model_ps(coefficients[6])
    print(torch.sum(result[0]))
    end = time.time()
    print(end - begin)

tensor(0.0771, device='cuda:0')
0.005642414093017578
tensor(0.0771, device='cuda:0')
0.0017838478088378906
tensor(0.0771, device='cuda:0')
0.001873016357421875
tensor(0.0771, device='cuda:0')
0.0017282962799072266
tensor(0.0771, device='cuda:0')
0.001875162124633789
tensor(0.0771, device='cuda:0')
0.001867055892944336
tensor(0.0771, device='cuda:0')
0.0018429756164550781
tensor(0.0771, device='cuda:0')
0.0018439292907714844
tensor(0.0771, device='cuda:0')
0.0018436908721923828
tensor(0.0771, device='cuda:0')
0.001848459243774414


In [34]:
model_bs = model_bs.cuda()

In [35]:
for _ in range(10):
    begin = time.time()
    result = model_bs(coefficients[1])
    print(torch.sum(result[0]))
    result = model_bs(coefficients[6])
    print(torch.sum(result[0]))
    end = time.time()
    print("time: ", end - begin)
    print(result[0].is_cuda)

tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.8396759033203125
True
tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.593317985534668
True
tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.5473482608795166
True
tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.5492579936981201
True
tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.5468900203704834
True
tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.548699140548706
True
tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.5508570671081543
True
tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.5488810539245605
True
tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.5485033988952637
True
tensor(0.0035, device='cuda:0')
tensor(0.0013, device='cuda:0')
time:  0.5508322715759277
True


In [36]:
print(torch.sum(result[0]))

tensor(0.0013, device='cuda:0')


In [37]:
print(result[0].shape)
print()

torch.Size([1, 111000, 300])



In [38]:
#model_bs_tensorrt = torch2trt(model_bs, [coefficients[1]])

In [39]:
print(result[0].is_cuda)

True


In [40]:
print(type(coefficients[1][1].device))

<class 'torch.device'>


In [41]:
print(str(coefficients[1][1].device) == 'cpu')

False


In [42]:
str(coefficients[1][1].device)

'cuda:0'

In [43]:
a = torch.FloatTensor([1, 2, 3])
print(str(a.device))

cpu


In [44]:
print(a.is_cuda)

False


In [45]:
print(coefficients[1][1].is_cuda)

True


In [46]:
from rascal.models import Kernel, SparsePoints, train_gap_model

In [47]:
HYPERS_PS_GRADIENTS = copy.deepcopy(HYPERS_PS)
HYPERS_PS_GRADIENTS['compute_gradients'] = True

In [48]:
soap_ps = SOAP(**HYPERS_PS_GRADIENTS)
begin = time.time()
ps = soap_ps.transform(nice.rascal_coefficients.process_structures(structures)).get_features(soap_ps)
end = time.time()
print(end - begin)
print(ps.shape)

0.24591517448425293
(1500, 450)


In [49]:
soap_ps = SOAP(**HYPERS_PS)
begin = time.time()
ps = soap_ps.transform(nice.rascal_coefficients.process_structures(structures)).get_features(soap_ps)
end = time.time()
print(end - begin)
print(ps.shape)

0.11308646202087402
(1500, 450)


In [50]:
SOAP()


TypeError: __init__() missing 5 required positional arguments: 'interaction_cutoff', 'cutoff_smooth_width', 'max_radial', 'max_angular', and 'gaussian_sigma_type'

In [ ]:
ob = soap_ps.transform(nice.rascal_coefficients.process_structures(structures))

In [ ]:
ob.managers

In [ ]:
HYPERS_GRAD = copy.deepcopy(HYPERS)
HYPERS_GRAD['compute_gradients'] = True

In [ ]:
 sph = SPH(**HYPERS)

In [ ]:
feat = sph.transform(nice.rascal_coefficients.process_structures(structures)).get_features(sph)

In [ ]:
print(feat.shape)

In [ ]:
ob = sph.transform(nice.rascal_coefficients.process_structures(structures))

In [ ]:
ob.get_features()